In [5]:
!pip install rlcard
!pip install torch==2.0.0 --user
!pip install rlcard[torch]


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/172.3 MB 2.3 MB/s eta 0:01:15
   ---------------------------------------- 0.3/172.3 MB 4.2 MB/s eta 0:00:41
   ---------------------------------------- 1.1/172.3 MB 8.7 MB/s eta 0:00:20
    --------------------------------------- 2.5/172.3 MB 14.6 MB/s eta 0:00:12
    --------------------------------------- 3.9/172.3 MB 17.8 MB/s eta 0:00:10
   - -------------------------------------- 6.4/172.3 MB 24.1 MB/s eta 0:00:07
   -- ------------------------------------- 9.2/172.3 MB 29.6 MB/s eta 0:00:06
   -- ------------------------------------- 10.2/172.3 MB 28.3 MB/s eta 0:00:06
   --- ------------------------------------ 13.6/172.3 MB 50.4 MB/s eta 0:00:04
   --- ------------------------------------ 14.8/172.3 MB 46.7 MB/s eta 0:00:04
   ---- ----------------------------------- 18.1/172.3 MB 50.4 MB/s eta 0:00:04
   ---- ----------------------------------- 21.0/172.3 MB 5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.0 requires torch==2.4.0, but you have torch 2.0.0 which is incompatible.
torchvision 0.19.0 requires torch==2.4.0, but you have torch 2.0.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
import torch
import time

In [95]:
#Include in step function of agent to run with output
def print_state(state, action_record):
    from rlcard.games.uno.card import UnoCard

    ''' Print out the state of a given player

    Args:
        player (int): Player id
    '''
    _action_list = []
    for i in range(1, len(action_record)+1):
        if action_record[-i][0] == state['current_player']:
            break
        _action_list.insert(0, action_record[-i])
    for pair in _action_list:
        print('>> Player', pair[0]+1, 'chooses ', end='')
        _print_action(pair[1])
        print('')

    curr_player = state['current_player'] + 1
    print(f'\n=============== Player {curr_player} ===============')
    UnoCard.print_cards(state['hand'])
    print('')
    print('=============== Table Card ===============')
    UnoCard.print_cards(state['target'], wild_color=True)
    print('')
    print('========== Players Card Number ===========')
    for i in range(state['num_players']):
        if i != state['current_player']:
            print('Player {} has {} cards.'.format(i+1, state['num_cards'][i]))
    print('======== Actions You Can Choose =========')
    for i, action in enumerate(state['legal_actions']):
        print(str(i)+': ', end='')
        UnoCard.print_cards(action, wild_color=True)
        if i < len(state['legal_actions']) - 1:
            print(', ', end='')
    print('\n')

def print_action(action):
    ''' Print out an action in a nice form

    Args:
        action (str): A string a action
    '''
    UnoCard.print_cards(action, wild_color=True)

In [98]:
class RandomAgent(object):
    ''' A random agent. Random agents is for running toy examples on the card games
    '''

    def __init__(self, num_actions):
        ''' Initilize the random agent

        Args:
            num_actions (int): The size of the ouput action space
        '''
        self.use_raw = False
        self.num_actions = num_actions

    @staticmethod
    def step(state):
        ''' Predict the action given the curent state in gerenerating training data.

        Args:
            state (dict): An dictionary that represents the current state

        Returns:
            action (int): The action predicted (randomly chosen) by the random agent
        '''
        print_state(state['raw_obs'], state['action_record'])
        time.sleep(2)
        return np.random.choice(list(state['legal_actions'].keys()))

    def eval_step(self, state):
        ''' Predict the action given the current state for evaluation.
            Since the random agents are not trained. This function is equivalent to step function

        Args:
            state (dict): An dictionary that represents the current state

        Returns:
            action (int): The action predicted (randomly chosen) by the random agent
            probs (list): The list of action probabilities
        '''
        probs = [0 for _ in range(self.num_actions)]
        for i in state['legal_actions']:
            probs[i] = 1/len(state['legal_actions'])

        info = {}
        info['probs'] = {state['raw_legal_actions'][i]: probs[list(state['legal_actions'].keys())[i]] for i in range(len(state['legal_actions']))}

        return self.step(state), info

In [99]:
''' A toy example of playing against rule-based bot on UNO
'''

import rlcard
from rlcard import models
from rlcard.agents.human_agents.uno_human_agent import HumanAgent, _print_action

# Make environment
env = rlcard.make('uno')
human_agent = HumanAgent(env.num_actions)
random_agent = RandomAgent(env.num_actions)
print(env.num_players)
env.set_agents([
    random_agent,
    random_agent,
])

print(">> UNO rule model V1")
print(">> Start a new game")

trajectories, payoffs = env.run()
for i in range(1, len(action_record)+1):
    if action_record[-i][0] == state['current_player']:
        break
    _action_list.insert(0, action_record[-i])
for pair in _action_list:
    print('>> Player', pair[0], 'chooses ', end='')
    _print_action(pair[1])
    print('')

print('===============     Result     ===============')
for i, player in enumerate(payoffs):
    if player > 0:
        print(f"Player {i+1} Won")
print('')


2
>> UNO rule model V1
>> Start a new game

=============== Player 1 ===============
Reverse, 2, Draw-2, Reverse, Skip, 5, 7
=============== Table Card ===============
0
========== Players Card Number ===========
Player 2 has 7 cards.
======== Actions You Can Choose =========
0: Draw-2, 1: 5

>> Player 1 chooses 5

=============== Player 2 ===============
4, Skip, 2, 5, 3, 7, 4
=============== Table Card ===============
5
========== Players Card Number ===========
Player 1 has 6 cards.
======== Actions You Can Choose =========
0: 5

>> Player 2 chooses 5

=============== Player 1 ===============
Reverse, 2, Draw-2, Reverse, Skip, 7
=============== Table Card ===============
5
========== Players Card Number ===========
Player 2 has 6 cards.
======== Actions You Can Choose =========
0: Reverse, 1: 7

>> Player 1 chooses 7

=============== Player 2 ===============
4, Skip, 2, 3, 7, 4
=============== Table Card ===============
7
========== Players Card Number ===========
Player 1 has 5 car